# 1. 라이브러리 로드

In [46]:
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import seaborn as sns

from statsmodels.tsa.seasonal import seasonal_decompose

import warnings
warnings.filterwarnings('ignore')

# 2. 데이터 로드

In [3]:
google_trend_df = pd.read_csv('/content/drive/MyDrive/sogang_thesis_prj/dataset/total_google_trend.csv', index_col=0)
google_trend_df['date'] = pd.to_datetime(google_trend_df['date'])
google_trend_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 16511 entries, 0 to 16510
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype         
---  ------  --------------  -----         
 0   date    16511 non-null  datetime64[ns]
 1   trend   16511 non-null  int64         
 2   artist  16511 non-null  object        
dtypes: datetime64[ns](1), int64(1), object(1)
memory usage: 516.0+ KB


In [4]:
google_trend_test_df = pd.read_excel('/content/drive/MyDrive/sogang_thesis_prj/dataset/test_google_trend.xlsx')
google_trend_test_df['date'] = pd.to_datetime(google_trend_test_df['date'])
google_trend_test_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 279 entries, 0 to 278
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype         
---  ------  --------------  -----         
 0   date    279 non-null    datetime64[ns]
 1   trend   279 non-null    int64         
 2   artist  279 non-null    object        
dtypes: datetime64[ns](1), int64(1), object(1)
memory usage: 6.7+ KB


In [5]:
artist_info_df = pd.read_csv('/content/drive/MyDrive/sogang_thesis_prj/dataset/top5_group_info.csv', index_col=0)
artist_info_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 4839 entries, 0 to 4838
Data columns (total 20 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   artist_name          4839 non-null   object 
 1   artist_id            4839 non-null   object 
 2   debut_date           4839 non-null   object 
 3   artist_types         4839 non-null   object 
 4   member_cnt           4839 non-null   int64  
 5   sub_group_cnt        4839 non-null   int64  
 6   label                4734 non-null   object 
 7   album_id             4839 non-null   object 
 8   album_name           4839 non-null   object 
 9   album_type           4839 non-null   object 
 10  release_date         4839 non-null   object 
 11  main_genre           0 non-null      float64
 12  sub_genre            0 non-null      float64
 13  track_cnt            4839 non-null   int64  
 14  avg_track_play_time  4749 non-null   float64
 15  participant_type     4839 non-null   object

In [13]:
# 앨범 데이터
album_info_df = artist_info_df.drop(['main_genre', 'sub_genre'], axis=1)[['album_name', 'album_type', 'artist_name', 'release_date', 'track_cnt', 'avg_track_play_time']].drop_duplicates()
album_info_df['release_date'] = pd.to_datetime(album_info_df['release_date'])

album_info_df = album_info_df[album_info_df['artist_name'] != 'NCT DREAM']

album_info_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 220 entries, 0 to 4715
Data columns (total 6 columns):
 #   Column               Non-Null Count  Dtype         
---  ------               --------------  -----         
 0   album_name           220 non-null    object        
 1   album_type           220 non-null    object        
 2   artist_name          220 non-null    object        
 3   release_date         220 non-null    datetime64[ns]
 4   track_cnt            220 non-null    int64         
 5   avg_track_play_time  210 non-null    float64       
dtypes: datetime64[ns](1), float64(1), int64(1), object(3)
memory usage: 12.0+ KB


# 3. 데이터 전처리

In [40]:
def preprocess_google_trend_df(trend_df):

    # 앨범 데이터 추가
    trend_df = trend_df.merge(album_info_df, how='left', left_on=['artist', 'date'], right_on=['artist_name', 'release_date']).drop('artist_name', axis=1)

    # 앨범 발매일 이후 7일 동일 데이터 적용
    # 앨범 초동 판매량 집계 기준일에 따라 7일 적용
    for i, row in trend_df.iterrows():

        artist = row['artist']
        release_date = row['release_date']
        end_date = release_date + pd.Timedelta(days=7)  # 발매일 이후 7일

        # 해당 기간에 데이터 할당
        mask = (trend_df['artist'] == artist) & \
              (trend_df['date'] >= release_date) & (trend_df['date'] < end_date)

        trend_df.loc[mask, 'release_date'] = row['release_date']
        trend_df.loc[mask, 'album_name'] = row['album_name']
        trend_df.loc[mask, 'album_type'] = row['album_type']

        trend_df.loc[mask, 'track_cnt'] = row['track_cnt']
        trend_df.loc[mask, 'avg_track_play_time'] = row['avg_track_play_time']

    trend_df[trend_df['album_name'].isnull() == False]

    # 앨범 장르 인코딩
    train_album_type_df = pd.get_dummies(trend_df['album_type'])
    trend_df = pd.concat([trend_df, train_album_type_df], axis=1)

    # 모든 bool 타입 열을 1/0으로 변환
    trend_df = trend_df.applymap(lambda x: int(x) if isinstance(x, bool) else x)
    trend_df = trend_df.drop(['album_type'], axis=1)

    # 최신 앨범 발매일 도달 기간
    trend_df['release_duration'] = (trend_df['date'] - trend_df['release_date']).dt.days

    return trend_df

In [59]:
def interpolate_trend_df(trend_df):

    trend_df = trend_df.set_index('date')

    # trend2: artist별 time interpolation & backward fill 수행
    # trend3: artist별 trend 지수 추세선 제거 적용
    google_trend_df_list = []
    for artist in trend_df['artist'].unique():

        single_artist_trend_df = trend_df.query(f"artist == '{artist}'")

        # trend3
        decomposition = seasonal_decompose(single_artist_trend_df['trend'], model='additive', period=12)
        trend = decomposition.trend

        single_artist_trend_df['trend3'] = single_artist_trend_df['trend'] - trend
        single_artist_trend_df['trend3'] = single_artist_trend_df['trend3'].interpolate(method='time').fillna(method='bfill')

        # trend2
        single_artist_trend_df.loc[single_artist_trend_df['trend'] == 0, 'trend'] = np.NaN # 0인 trend 값 interpolation 수행
        single_artist_trend_df['trend2'] = single_artist_trend_df['trend'].interpolate(method='time').fillna(method='bfill')

        google_trend_df_list.append(single_artist_trend_df)

    google_trend_df = pd.concat(google_trend_df_list).reset_index().fillna(0) # 나머지 앨범 데이터에 대하여 0으로 결측치 처리

    return google_trend_df

In [43]:
# 아티스트 별 훈련 데이터 기간 통일
# ML 버전에서 사용한 기간에 맞게 모두 2023년 6월 30일 ~ 2024년 6월 30일 동안의 데이터 가져옴
train_df = google_trend_df[google_trend_df['date'].isin([date for date in pd.date_range(start='2023-06-30', end='2024-06-30')])].reset_index(drop=True)

# train 기간 nct dream 0인 trend 값 280일 ==> 해당 케이스 제거
train_df[train_df['trend'] == 0].groupby('artist').count()['date']
train_df = train_df[train_df['artist'] != 'NCT DREAM']

train_df = preprocess_google_trend_df(train_df)
train_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3307 entries, 0 to 3306
Data columns (total 12 columns):
 #   Column               Non-Null Count  Dtype         
---  ------               --------------  -----         
 0   date                 3307 non-null   datetime64[ns]
 1   trend                3307 non-null   int64         
 2   artist               3307 non-null   object        
 3   album_name           308 non-null    object        
 4   release_date         308 non-null    datetime64[ns]
 5   track_cnt            308 non-null    float64       
 6   avg_track_play_time  308 non-null    float64       
 7   Album                3307 non-null   int64         
 8   EP                   3307 non-null   int64         
 9   Other                3307 non-null   int64         
 10  Single               3307 non-null   int64         
 11  release_duration     308 non-null    float64       
dtypes: datetime64[ns](2), float64(3), int64(5), object(2)
memory usage: 310.2+ KB


In [63]:
interpolate_train_df = interpolate_trend_df(train_df).drop(['album_name', 'release_date'], axis=1)
interpolate_train_df

,date,trend,artist,track_cnt,avg_track_play_time,Album,EP,Other,Single,release_duration,trend3,trend2
0,2023-06-30,21.0,aespa,0.0,0.0,0,0,0,0,0.0,5.708333,21.0
1,2023-07-01,28.0,aespa,0.0,0.0,0,0,0,0,0.0,5.708333,28.0
2,2023-07-02,19.0,aespa,0.0,0.0,0,0,0,0,0.0,5.708333,19.0
3,2023-07-03,23.0,aespa,0.0,0.0,0,0,0,0,0.0,5.708333,23.0
4,2023-07-04,17.0,aespa,0.0,0.0,0,0,0,0,0.0,5.708333,17.0
...,...,...,...,...,...,...,...,...,...,...,...,...
3302,2024-06-26,24.0,TWICE,0.0,0.0,0,0,0,0,0.0,-5.333333,24.0
3303,2024-06-27,23.0,TWICE,0.0,0.0,0,0,0,0,0.0,-5.333333,23.0
3304,2024-06-28,20.0,TWICE,0.0,0.0,0,0,0,0,0.0,-5.333333,20.0
3305,2024-06-29,34.0,TWICE,0.0,0.0,0,0,0,0,0.0,-5.333333,34.0


In [45]:
# test기간은 2024년 7월 1일 ~ 2024년 7월 31일
test_df = google_trend_test_df[google_trend_test_df['artist'] != 'NCT DREAM'].copy()
test_df = preprocess_google_trend_df(test_df)
test_df['EP'] = 0; test_df['Other'] = 0
test_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 279 entries, 0 to 278
Data columns (total 12 columns):
 #   Column               Non-Null Count  Dtype         
---  ------               --------------  -----         
 0   date                 279 non-null    datetime64[ns]
 1   trend                279 non-null    int64         
 2   artist               279 non-null    object        
 3   album_name           21 non-null     object        
 4   release_date         21 non-null     datetime64[ns]
 5   track_cnt            21 non-null     float64       
 6   avg_track_play_time  14 non-null     float64       
 7   Album                279 non-null    int64         
 8   Single               279 non-null    int64         
 9   release_duration     21 non-null     float64       
 10  EP                   279 non-null    int64         
 11  Other                279 non-null    int64         
dtypes: datetime64[ns](2), float64(3), int64(5), object(2)
memory usage: 26.3+ KB


In [64]:
interpolate_test_df = interpolate_trend_df(test_df).drop(['album_name', 'release_date'], axis=1)
interpolate_test_df

,date,trend,artist,track_cnt,avg_track_play_time,Album,Single,release_duration,EP,Other,trend3,trend2
0,2024-07-01,76.0,aespa,0.0,0.000000,0,0,0.0,0,0,3.083333,76.0
1,2024-07-02,87.0,aespa,3.0,194914.333333,0,1,0.0,0,0,3.083333,87.0
2,2024-07-03,100.0,aespa,3.0,194914.333333,0,1,1.0,0,0,3.083333,100.0
3,2024-07-04,96.0,aespa,3.0,194914.333333,0,1,2.0,0,0,3.083333,96.0
4,2024-07-05,67.0,aespa,3.0,194914.333333,0,1,3.0,0,0,3.083333,67.0
...,...,...,...,...,...,...,...,...,...,...,...,...
274,2024-07-27,46.0,TWICE,0.0,0.000000,0,0,0.0,0,0,0.708333,46.0
275,2024-07-28,50.0,TWICE,0.0,0.000000,0,0,0.0,0,0,0.708333,50.0
276,2024-07-29,43.0,TWICE,0.0,0.000000,0,0,0.0,0,0,0.708333,43.0
277,2024-07-30,47.0,TWICE,0.0,0.000000,0,0,0.0,0,0,0.708333,47.0


# 3. 모델 구성
1. 각 아티스트 별 데이터 분할
2. Sequential Dataset 생성 (훈련 데이터에 한함)
3. 훈련 데이터를 훈련 & 검증으로 분할 (8:2)
4. 훈련, 검증, test 데이터 피처 스케일링 (TM 방식에서는 적용 안한 상태)
5. 모델 훈련
6. test 데이터로 최종 모델 예측값 확인